In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

base_model = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")
model = AutoModelForSequenceClassification.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16)

model = model.eval()

In [11]:
tokens = tokenizer(["Fuck this economy. I hate aig and their non loan given asses"], return_tensors='pt',
                           padding=True)
output = model(**tokens)

output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
output.argmax()

tensor(0)

In [12]:
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd

def map_output(arg):
  # Stars 0 and 1 => Negative (which is 0)
  # Stars 2 => Neutral (which is 2)
  # Stars 3 and 4 => Positive (which is 4)
  dic = {0: 0, 1: 4}
  return dic[arg]


def test_sentiment140(model, tokenizer, batch_size=8):
    dataset = load_dataset('sentiment140')
    dataset = dataset["test"]
    dataset = dataset.to_pandas()

    negative_df = dataset.query("sentiment == 0")[:50]
    positive_df = dataset.query("sentiment == 4")[:50]

    dataset = pd.concat([negative_df, positive_df])

    dataset = dataset.rename(columns={"sentiment": "target"})


    dataset = dataset[['text', 'target']]

    # print example
    print(f"\n\nPrompt example:\n{dataset['text'][1]}\n\n")

    context = dataset['text'].tolist()
    total_steps = dataset.shape[0]//batch_size + 1
    print(
        f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text = []

    for i in tqdm(range(total_steps)):
        tmp_context = context[i * batch_size:(i+1) * batch_size]

        tokenizer.pad_token = "[PAD]"

        tokens = tokenizer(tmp_context, return_tensors='pt',
                           padding=True)

        output = model(**tokens)
        output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
        out_text.append(output.detach().numpy())
        torch.cuda.empty_cache()

    out_text = [item for sublist in out_text for item in sublist]
    dataset["out_text"] = out_text
    dataset["new_out"] = dataset["out_text"].apply(np.argmax).apply(map_output)

    acc = accuracy_score(dataset["target"], dataset["new_out"])
    f1_macro = f1_score(dataset["target"], dataset["new_out"], average="macro")
    f1_micro = f1_score(dataset["target"], dataset["new_out"], average="micro")
    f1_weighted = f1_score(
        dataset["target"], dataset["new_out"], average="weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    return dataset

dataset = test_sentiment140(model, tokenizer)
dataset



Prompt example:
Reading my kindle2...  Love it... Lee childs is good read.


Total len: 100. Batchsize: 8. Total steps: 13


100%|██████████| 13/13 [00:00<00:00, 38.44it/s]

Acc: 0.86. F1 macro: 0.8590982286634461. F1 micro: 0.8599999999999999. F1 weighted (BloombergGPT): 0.859098228663446. 


,text,target,out_text,new_out
6,Fuck this economy. I hate aig and their non lo...,0,"[0.9989183, 0.0010817152]",0
11,@Karoli I firmly believe that Obama/Pelosi hav...,0,"[0.94509614, 0.054903854]",0
14,"dear nike, stop with the flywire. that shit is...",0,"[0.9997305, 0.00026947915]",0
16,I was talking to this guy last night and he wa...,0,"[0.2018919, 0.7981081]",4
18,"@ludajuice Lebron is a Beast, but I'm still ch...",0,"[0.0021157034, 0.99788433]",4
...,...,...,...,...
106,"@psychemedia I really liked @kswedberg's ""Lear...",4,"[0.0016613906, 0.9983386]",4
108,"Very Interesting Ad from Adobe by Goodby, Silv...",4,"[0.0053536464, 0.9946464]",4
109,Goodby Silverstein agency new site! http://www...,4,"[0.0006838353, 0.9993162]",4
110,"RT @designplay Goodby, Silverstein's new site:...",4,"[0.0002173914, 0.9997826]",4


In [13]:
dataset["text"].iloc[0]

'Fuck this economy. I hate aig and their non loan given asses.'

In [14]:
path = '../../results/general/distilbert.csv'

In [15]:
dataset.to_csv(path)

In [16]:
import pandas as pd

df = pd.read_csv(path, index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.869, Recall: 0.86, F1: 0.859, Accuracy: 0.86
